In [1]:
import fitz  # PyMuPDF
import os
from dotenv import load_dotenv
from apify_client import ApifyClient

load_dotenv()

False

In [ ]:
# Load LLM Model


In [ ]:
# Load Apify api

In [ ]:
# Extract text from PDF
def extract_text_from_pdf(uploaded_file):
    doc = fitz.open(stream=uploaded_file.read(), filetype='pdf')
    text = ''
    for page in doc:
        text += page.get_text()
    return text

In [ ]:
# Ask LLAMA to generate output
def ask_llama(prompt, max_tokens=500):
    response = llama_client.generate_completion(prompt=prompt, temperature=0.5)
    if isinstance(response, dict) and 'choices' in response:
        return response['choices'][0]['message']['content']
    return response

In [ ]:
# Fetch Linkedin jobs using Apify
def fetch_linkedin_jobs(search_query, location="Sri Lanka", rows=60):
    run_input = {
        'title': search_query,
        'location': location,
        'rows': rows,
        'proxy': {
            'useApifyProxy': True,
            'apifyProxyGroups': ['RESIDENTIAL']
        }
    }
    run = apify_client.actor('api_key_for_apify').call(run_input=run_input)
    jobs = list(apify_client.dataset(run['defaultDatasetId']).iterate_items())
    return jobs

In [ ]:
# Fetch Naukri jobs using Apify
def fetch_naukri_jobs(search_query, location="Sri Lanka", rows=60):
    run_input = {
        'keyword': search_query,
        'maxJobs': 60,
        'freshness': 'all',
        'sortBy': 'relevance',
        'experience': 'all'
    }
    run = apify_client.actor('api_key_for_apify').call(run_input=run_input)
    jobs = list(apify_client.dataset(run['defaultDatasetId']).iterate_items())
    return jobs